# Self study 7

In this self study we start to investigate some community detection (graph clustering) techniques. As toy datasets we use the Lazega Law and the Zachary Karate club networks. Some of the following code was already used in self study 4.

In [ ]:
import numpy as np
import networkx as nx
import scipy as sp
from networkx.algorithms.community import kernighan_lin_bisection
from networkx.algorithms.community.centrality import girvan_newman

In [ ]:
def get_att_array(G,att_name):
    ret_array=np.zeros(nx.number_of_nodes(G))
    for i,n in enumerate(G.nodes()):
        ret_array[i]=G.nodes[n][att_name]
    return(ret_array)

Remember that the edges in the Lazega network represent a directed 'friendship' relation, and that nodes have attributes Practice, Age, Seniority, Office, Gender and Status. For the purpose of this self study we turn the directed graph into an undirected one.

In the following you can see that already the 'Kamada Kawai' layout algorithm separates the nodes somewhat according tho the office location:

In [ ]:
lazega=nx.readwrite.graphml.read_graphml('lazega.gml')
lazega=lazega.to_undirected()
nx.draw_kamada_kawai(lazega,with_labels=True,node_color=get_att_array(lazega,'Office'))

We next load the Zachary network. This is a built-in of networkx:

In [ ]:
zachary = nx.karate_club_graph()

This does not contain the information on the two "ground truth" communities. This is here manually constructed:

In [ ]:
zachary_gt=np.array([0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1])

Using a generic drawing method, we obtain a layout that separates the two ground truth communities quite well:

In [ ]:
nx.draw(zachary,with_labels=True,node_color=zachary_gt)

**Task 1:** Use the networkx implementations of the Kernighan-Lin and Newman-Girvan algorithms to divide the Lazega and Zachary networks into 2,3,4 communities. Compare the communities returned by the algorithms with ground truth clusters in the network: the ground truth communities in Zachary, and the clusters defined by node attributes Office, Gender, Practice, Status in Lazega.

**Task 2:** (a bit more involved; consider this as optional): construct a networkx graph out of the web-pages you have crawled in self study 1: each crawled page is a node, and edges are defined by hyperlinks. These are directed edges, but we can also make them undirected for community detection. Apply the Kernighan-Lin and Newman-Girvan algorithms to your crawled web data. Do you see a correspondence between the communities that are returned, and what you know about the pages (urls, content, ...)? 